In [1]:
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans

In [2]:
SEED = 2024
k = 3

In [3]:
dataset = pd.read_csv('land_transport_dataset.csv', sep=';')
dataset.shape

(91684, 22)

In [4]:
dataset.columns

Index(['agente_causador', 'cd_cbo', 'cbo_descricao', 'cnae_descricao',
       'emitente', 'especie_beneficio', 'filiacao_segurado', 'indica_obito',
       'municipio_empregador', 'natureza_lesao', 'parte_corpo_atingida',
       'sexo', 'tp_acidente', 'uf_empregador', 'dt_acidente',
       'cd_municipio_empregador', 'idade', 'cid_grupo', 'cid_capitulo',
       'cbo_subgrupo_principal', 'dt_mes_acidente', 'dt_ano_acidente'],
      dtype='object')

Based on the metrics evaluated, not considering a time atribute had the best result as presentede in the file evaluation_n_clusters

Silhouette score: 0.0718142555343571 (The highest, the best)

Davies Bouldin Score: 3.6082661982841486 (Te lowest, the best)

Calinski Harabasz Score: 3848.796463847226 (The highest, the best)

In [5]:
colunas_semTempo = ['agente_causador','cnae_descricao','emitente', 'especie_beneficio', 'filiacao_segurado', 'indica_obito', 'cid_grupo',
               'natureza_lesao', 'parte_corpo_atingida','sexo', 'tp_acidente', 'uf_empregador', 'idade','cbo_subgrupo_principal']

In [6]:
df = dataset[colunas_semTempo]
df_d = pd.get_dummies(df)
df_d.shape

(91684, 605)

#### Transforming each line in a sum of 1, cos distance

In [7]:
X = df_d.div(df_d.sum(axis=1), axis='rows')

In [8]:
km = KMeans(n_clusters = k, max_iter=10_000, n_init=100, random_state=SEED)

In [9]:
X_t = km.fit_predict(X)

In [10]:
centroids = km.cluster_centers_
max_ = len(centroids[0])
lista = []
for i in range(max_):
    lista.append([X.columns.values[i], "{:.6f}".format(centroids[:,i].var())])
    
alta_var = pd.DataFrame(lista).sort_values(by=1, ascending=False)
alta_var.columns = ['desc', 'var']
alta_var['var'] = pd.to_numeric(alta_var['var'], errors='coerce')
alta_var.nlargest(20, 'var')

,desc,var
529,tp_acidente_Trajeto,0.001127
530,tp_acidente_Típico,0.001080
286,cnae_descricao_Transporte Rodoviario de Carga,0.000874
285,cnae_descricao_Transporte Rodoviario Coletivo ...,0.000454
590,cbo_subgrupo_principal_Trabalhadores De Funçõe...,0.000102
167,"agente_causador_Motocicleta, Motoneta",0.000073
284,cnae_descricao_Transporte Metroferroviario de ...,0.000043
473,natureza_lesao_Fratura,0.000039
27,"agente_causador_Ataque de Ser Vivo, Nic",0.000030
504,parte_corpo_atingida_Mao (Exceto Punho ou Dedos),0.000025


In [13]:
df_d.loc[:, 'clus'] = X_t

In [14]:
df_cluster_count = df_d.pivot_table(index = 'clus', aggfunc='sum')
df_cluster_count.loc[:,'Total'] = df_cluster_count.sum(numeric_only=True, axis=1)
df_cluster_count.loc['Total'] = df_cluster_count.sum(numeric_only=True, axis=0)

In [15]:
df_cluster_count = df_cluster_count.style
df_cluster_count = df_cluster_count.set_sticky(axis='index')
df_cluster_count = df_cluster_count.format('{:,.0f}')
headers = {
    'selector': 'th:not(.col_heading)',
    'props': 'color: black;'
}
df_cluster_count.set_table_styles([headers], overwrite=False)

In [16]:
df_cluster_perc = df_d.pivot_table(index = 'clus', aggfunc='sum')
df_cluster_perc = df_cluster_perc/df_cluster_perc.sum(numeric_only=True, axis=0)
df_cluster_perc.loc['Total'] = df_d.pivot_table(index = 'clus', aggfunc='sum').sum(numeric_only=True, axis=0)/df.shape[0]
df_cluster_perc = df_cluster_perc * 100

In [17]:
slice_ = df_cluster_perc[:-1]
slice_ = pd.IndexSlice[slice_.index, slice_.columns]

In [18]:
headers = {
    'selector': 'th:not(.col_heading)',
    'props': 'color: black;'
}
df_cluster_perc.style\
    .highlight_max(color='darkgreen', subset=slice_)\
    .highlight_min(color='darkred', subset=slice_)\
    .format('{:.3f}%')\
    .set_sticky(axis='index')\
    .set_table_styles([headers], overwrite=False)

In [19]:
df_cluster_sort = df_cluster_perc.sort_values('Total', axis=1, ascending=False)

In [20]:
headers = {
    'selector': 'th:not(.col_heading)',
    'props': 'color: black;'
}
df_cluster_sort.style\
    .highlight_max(color='darkgreen', subset=slice_)\
    .highlight_min(color='darkred', subset=slice_)\
    .format('{:.3f}%')\
    .set_sticky(axis='index')\
    .set_table_styles([headers], overwrite=False)

- Tanto o trajeto quanto típico entram para cid de acidentes
- parte do corpo e lesões multiplas bem divida entre tipico e trajeto
- nae_descricao_Transporte Rodoviario de Taxi 44,9% Trajeto e 36,96% Doença| 0.396%

### Cluster 0 Qtd: 254,898
CNAE Transporte Rodoviario Coletivo de Passageiros 69,76% | 20,56% <br>
CNAE Transporte Metroferroviario de Passageiros 78,74% | 5.41% <br>
cnae_descricao_Transporte Escolar: 45.714% | 0.267% <br>
cnae_descricao_Transporte Ferroviario de Carga: 62,24% | 1,27% <br>
cnae_descricao_Transporte Dutoviario: 74.031% | 0,281%

uf_empregador_Distrito Federal: 42.90% | 1.11%%
emitente_Sindicato: 67.756% | 0.768%

Natureza Lesao Doenca, Nic: 82,93% | 2,31%
Agente Causador - Ataque de Ser Vivo, Nic: 87,22% |3,90%
parte_corpo_atingida_Sistema Nervoso: 90.751% | 2.076%
uf_empregador_Amazonas: 60.963% | 2.040%

Indica öbito sim: 8,79% | 1,48%
Tipo Acidente Doença: 51,04% | 1,37% <br>
cid_grupo_Transt neuróticos, relacion stress e somatoformes : 90.494%: 1.434%
cid_grupo_Agressões 56.923% | 0.354%

Idade mista crescendo quanto mais velho 
(, 20.0]: 8,32% | 3,89% <br>
(20.0, 30.0]: 12,49% | 23,89% <br>
(30.0, 40.0]: 19,43% | 29,51% <br>
(40.0, 50.0]: 24,26% | 24,95% <br>
(50.0, 60.0]: 27,16%% | 14,42% <br>
(60.0, inf]: 25,64% | 3,34%

### Cluster 1 Qtd: 303,368
Tipo Acidente Trajeto: 99,70% | 23,70% <br> 
Agente causador Motocicleta, Motoneta: 86,32% | 7.36%
agente_causador_Rua e Estrada - Superficie Utilizada para Sus: 57,84%
gente_causador_Animal Vivo: 41.297%	 0.909%| 
agente_causador_Bicicleta: 91.045%	: 0.512%

Indica öbito sim: 32,57% | 1,48%

cbo_subgrupo_principal_Trabalhadores De Atendimento Ao Público: 54.396% | 0.397%
cbo_subgrupo_principal_Técnicos De Nível Médio Das Ciências Físicas, Químicas, Engenharia E Afins: 54.727% | 0.819%
cbo_subgrupo_principal_Técnicos De Nivel Médio Nas Ciências Administrativas: 53.168% | 0.396%

Idade mista decrescendo quanto mais velho 
(, 20.0]: 28,22% | 3,89% <br>
(20.0, 30.0]: 25,72% | 23,89% <br>
(30.0, 40.0]: 25,06% | 29,51% <br>
(40.0, 50.0]: 22,46% | 24,95% <br>
(50.0, 60.0]: 19,67%% | 14,42% <br>
(60.0, inf]: 17,23% | 3,34%

### Cluster 2 Qtd: 724,609
Tipo Acidente Tipico: 74,52% | 74,965% <br> 
CNAE Transporte Rodoviario de Carga: 78,35% | 71,81%<br>

Filiação Empregado: 56,49% | 99,88% <br>
Emitente Empregado: 56,88% | 97,63% <br>
Benificio Pa: 56,50% | 99.20% <br>
Indica Óbito> 56,44% | 98.82%  -  sim: 58,64% | 1,48%<br>
Sexo Masculino: 58,97%  | 97,764% <br>

CBO Trabalhadores De Funções Transversais 65,47% | 57,66%<br>
UF Empregador São Paulo: 53,79% | 35,55% <br>
Natureza Lesao Fratura: 53,97% | 26,30% <br>
natureza_lesao_Outras Lesoes, Nic: 53,17% | 3,27%  
cid_grupo_Dorsopatias : 60,74%	 | 2,98% - Questões ergonomicas

Idade mista descrescendo quanto mais velho 
(, 20.0]: 63,45% | 3,89% <br>
(20.0, 30.0]: 61,79% | 23,89% <br>
(30.0, 40.0]: 55,50% | 29,51% <br>
(40.0, 50.0]: 53,27% | 24,95% <br>
(50.0, 60.0]: 53,17% | 14,42% <br>
(60.0, inf]: 57.13% | 3,34%

In [21]:
clusters = pd.DataFrame(df.clus.value_counts())
clusters.columns = ['Qtd']
clusters['Perc'] = clusters['Qtd']/df.shape[0]*100

In [22]:
clusters.style.format({'Qtd':'{:,}', 'Perc': '{:.2f}%'})

,Qtd,Perc
2,"51,779",56.48%
1,"21,689",23.66%
0,"18,216",19.87%
